In [1]:
# data.ipynb
import pandas as pd
import numpy as np

# health metrics
data = {
    'timestamp': pd.date_range(start='2023-10-01', periods=100, freq='min'),
    'heart_rate': np.random.randint(60, 100, 100),
    'blood_oxygen': np.random.randint(90, 100, 100),
    'activity_level': np.random.choice(['low', 'moderate', 'high'], 100)
}

df = pd.DataFrame(data)
df.to_csv('health_data.csv', index=False)
print(df.head())


            timestamp  heart_rate  blood_oxygen activity_level
0 2023-10-01 00:00:00          88            97       moderate
1 2023-10-01 00:01:00          91            91           high
2 2023-10-01 00:02:00          61            92            low
3 2023-10-01 00:03:00          81            90            low
4 2023-10-01 00:04:00          80            94            low


In [4]:
# model_training.ipynb
from sklearn.ensemble import IsolationForest
import pandas as pd
import joblib

df = pd.read_csv('health_data.csv')

model = IsolationForest(contamination=0.1)
df['anomaly'] = model.fit_predict(df[['heart_rate', 'blood_oxygen']])
df['anomaly'] = df['anomaly'].apply(lambda x: 'Anomaly' if x == -1 else 'Normal')

joblib.dump(model, 'anomaly_model.pkl') 

print(df[['timestamp', 'heart_rate', 'blood_oxygen', 'anomaly']].head())


             timestamp  heart_rate  blood_oxygen anomaly
0  2023-10-01 00:00:00          88            97  Normal
1  2023-10-01 00:01:00          91            91  Normal
2  2023-10-01 00:02:00          61            92  Normal
3  2023-10-01 00:03:00          81            90  Normal
4  2023-10-01 00:04:00          80            94  Normal


In [6]:
from flask import Flask, render_template
import pandas as pd
import joblib

app = Flask(__name__)

@app.route('/')
def home():
    df = pd.read_csv('health_data.csv')
    model = joblib.load('anomaly_model.pkl')
    df['anomaly'] = model.predict(df[['heart_rate', 'blood_oxygen']])
    df['anomaly'] = df['anomaly'].apply(lambda x: 'Anomaly' if x == -1 else 'Normal')
    data = df.tail(1).to_dict('records')[0]
    return render_template('index.html', data=data)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

<!DOCTYPE html>
<html>
<head>
    <title>Health Dashboard</title>
</head>
<body>
    <h2>Latest Health Reading</h2>
    <ul>
        <li>Heart Rate: {{ data['heart_rate'] }}</li>
        <li>Blood Oxygen: {{ data['blood_oxygen'] }}</li>
        <li>Activity Level: {{ data['activity_level'] }}</li>
        <li>Status: {{ '⚠️ ' + data['anomaly'] if data['anomaly'] == 'Anomaly' else '✅ Normal' }}</li>
    </ul>
</body>
</html>
